<a href="https://colab.research.google.com/github/NikolaiZolotykh/ScientificPython/blob/master/16.02.%20%D0%A6%D0%9B%D0%9F.%20%D0%97%D0%B0%D0%B4%D0%B0%D1%87%D0%B0%20%D1%81%D0%BE%D1%81%D1%82%D0%B0%D0%B2%D0%BB%D0%B5%D0%BD%D0%B8%D1%8F%20%D1%80%D0%B0%D1%81%D0%BF%D0%B8%D1%81%D0%B0%D0%BD%D0%B8%D1%8F.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pulp
import pulp

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.3/14.3 MB 71.9 MB/s eta 0:00:00


In [ ]:
m = 3 # количество учебных дисциплин (преподавателей)
n = 3 # количество академических групп
p = 4 # количество тактов времени

In [ ]:
#m = 10 # количество учебных дисциплин (преподавателей)
#n = 10 # количество академических групп
#p = 12 # количество тактов времени

In [ ]:
I = range(m) # множество учебных дисциплин
J = range(n) # множество академических групп
T = range(p) # множество тактов


In [ ]:
# x[i][j][k] = 1 означает, что в k-й такт у j-й группы проходит занятие по i-й дисциплине */
x = pulp.LpVariable.dicts("x", (I, J, T), 0, 1, pulp.LpInteger)

In [ ]:
x[0][2][2]

x_0_2_2

In [ ]:
prob = pulp.LpProblem("School Scheduling Problem", pulp.LpMinimize)
#prob += 0, "Arbitrary Objective Function" 

/usr/local/lib/python3.8/dist-packages/pulp/pulp.py:1352: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


In [ ]:
# У каждой группы по каждой дисциплине должно пройти ровно одно занятие:
for i in I:
    for j in J:
        prob += pulp.lpSum([x[i][j][t] for t in T]) == 1

In [ ]:
# в любой такт по каждой дисциплине проходит не более одного занятия во всех группах:
for i in I:
    for t in T:
        prob += pulp.lpSum([x[i][j][t] for j in J]) <= 1

In [ ]:
# в любой такт у любой группы проходит не более одного занятия:
for j in J:
    for t in T:
        prob += pulp.lpSum([x[i][j][t] for i in I]) <= 1

In [ ]:
# 1-й преподаватель не может проводить занятия 4-й парой:
for j in J:
    prob += x[0][j][3] == 0

# 2-й преподаватель не может проводить занятия 3-й парой:
for j in J:
    prob += x[1][j][2] == 0

# 2-я группа не может учиться 1-й парой:
for i in I:
    prob += x[i][1][0] == 0 

# 3-я группа не может учиться 4-й парой    
for i in I:
    prob += x[i][2][3] == 0

In [ ]:
prob.solve()
print("Status:", pulp.LpStatus[prob.status])


Status: Optimal


In [ ]:
print("\n  Solution:\n\n \\\n  \\   Groups\n t \\\n    \\")
print("     ", end="")
for j in J:
    print("{:2d}".format(j), end="")
print()

for t in T:
    print("{:2d}   ".format(t), end="")
    for j in J:
        lesson = False
        for i in I:
            if pulp.value(x[i][j][t]) == 1:
                print("{:2d}".format(i), end="")
                lesson = True
                break
        if not lesson:
            print(" .", end="")    
    print()
print()



  Solution:

 \
  \   Groups
 t \
    \
      0 1 2 3 4 5 6 7 8 9
 0    2 . 7 5 1 0 4 6 8 9
 1    7 6 8 9 3 . 0 1 4 5
 2    3 2 9 8 6 4 7 . 5 0
 3    . 5 . 4 2 8 3 7 6 .
 4    . 7 4 . 9 5 2 3 1 6
 5    0 9 5 6 . 2 1 8 3 7
 6    4 8 6 1 0 7 5 9 . 3
 7    6 1 2 0 8 . 9 5 7 4
 8    9 . . 3 . 1 . 2 0 8
 9    8 3 0 . 4 9 6 . 2 1
10    1 0 3 7 5 6 8 4 . 2
11    5 4 1 2 7 3 . 0 9 .

